In [1]:
%matplotlib widget

import numpy as np

# Mapping to Feasible Space

1. load data
2. process
3. plot

## 1. Load data

x-y-theta

In [2]:
data3d = np.load('fake_data.npy')
print(data3d.shape)
data3d

(1575,)


array([(-90.,  0.,  0., 1.2918387e-04), (-90.,  0.,  1., 0.0000000e+00),
       (-90.,  0.,  2., 0.0000000e+00), ...,
       ( 90., 14., 12., 7.5058979e-01), ( 90., 14., 13., 5.8899295e-01),
       ( 90., 14., 14., 3.6736864e-01)],
      dtype=[('theta', '<f4'), ('x', '<f4'), ('y', '<f4'), ('manip', '<f4')])

In [3]:
def extract_specific_theta(data3d, theta):
    filter_arr = (data3d['theta'] == theta)
    rec = data3d[filter_arr]
    new_shape = rec.shape + (-1,)  # (225, -1) = (225,) + (-1,)
    normal_arr = rec.view(np.float32).reshape(new_shape)

    # ROW = 0
    COL = 1
    index = 0
    normal_arr = np.delete(normal_arr, index, COL)
    normal_arr = np.transpose(normal_arr)
    return normal_arr

In [4]:
theta_list = set(data3d['theta'])
manip_map = {}
for theta in theta_list:
    manip_map[theta] = extract_specific_theta(data3d, theta)

print(manip_map[0].shape)
print(manip_map.keys())

(3, 225)
dict_keys([0.0, -30.0, -60.0, -90.0, 90.0, 60.0, 30.0])


## 2. Process
### (1) Rotation Function

```
[rot]      | [theta0]
c0 -s0  0  | x1 x2 x3 x4 x5 ... x100
s0  c0  0  | y1 y2 y3 y4 y5 ... y100
 0   0  1  | u1 u2 u3 u4 u5 ... u100

```

In [5]:
def rotation_z(degree):
    rad = np.radians(degree)
    c, s = np.cos(rad), np.sin(rad)
    R = np.array((
        (c, -s, 0),
        (s,  c, 0),
        (0,  0, 1)
    ))
    return R

def manipulability_to_feasibility(manipulability_map):
    """
    manipulability_map & feasibility_map
        key: degree
        value: [(x, y, manip), ...]
    
    [rot]      | [theta0]
    c0 -s0  0  | x1 x2 x3 x4 x5 ... x100
    s0  c0  0  | y1 y2 y3 y4 y5 ... y100
     0   0  1  | u1 u2 u3 u4 u5 ... u100
    """
    feasibility_map = {}
    for theta, mlayer in manipulability_map.items():
        R = rotation_z(180 - theta)
        # print(type(R), R.shape)
        # print(type(mlayer), mlayer.shape)
        flayer = np.dot(R, mlayer)
        feasibility_map[theta] = flayer
    return feasibility_map

### (2) Linear Transformation

In [6]:
feasi_map = manipulability_to_feasibility(manip_map)
print(manip_map[0].shape)
print(feasi_map[0].shape)

(3, 225)
(3, 225)


## 3. Plot
### 2D plot

In [7]:
import matplotlib.pyplot as plt

def scatter_2d(ax, theta, xs, ys, manip, dot_size=1, arrow_size=2):
    colors = np.array([(1.0-m, m, 0.0, m) for m in manip])
    ax.scatter(xs, ys, c=colors, s=dot_size)
    ax.set_xlabel('x(cm)', fontsize=15)
    ax.set_ylabel('y(cm)', fontsize=15)
    ax.set_title('Manipulability Layer: $\Theta$ = %.1f deg'% theta)
    ax.grid(True)
    # robot and object
    arrow_x = np.array([1.0, -0.5, -0.5, 1.0]) * arrow_size
    arrow_y = np.array([0.0, 0.5, -0.5, 0.0]) * arrow_size
    ax.plot(arrow_x, arrow_y)
    

In [8]:
M = manip_map[0.0]
F = feasi_map[0.0]

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
scatter_2d(ax1, 0, M[0], M[1], M[2])
scatter_2d(ax2, 0, F[0], F[1], F[2])

xyminmax = [-20, 20, -20, 20]
ax1.axis(xyminmax)
ax2.axis(xyminmax)
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 3D plot

In [11]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def scatter_3d(ax, theta, xs, ys, manip, dot_size=1, arrow_size=2):
    colors = np.array([(1.0-m, m, 0.0, m) for m in manip])
    ax.scatter(xs, ys, zs=theta, c=colors, s=dot_size)
    ax.set_xlabel('x(cm)', fontsize=15)
    ax.set_ylabel('y(cm)', fontsize=15)
    ax.set_zlabel('$\Theta$(deg)', fontsize=15)
    ax.set_title('Manipulability Map')
    # robot and object
    arrow_x = np.array([1.0, -0.5, -0.5, 1.0]) * arrow_size
    arrow_y = np.array([0.0, 0.5, -0.5, 0.0]) * arrow_size
    arrow_z = np.array([theta, theta, theta, theta])
    ax.plot(arrow_x, arrow_y, arrow_z)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# scatter_3d(ax, data3d['theta'], data3d['x'], data3d['y'], data3d['manip'])
# fig.tight_layout()
# plt.show()

In [12]:
fig = plt.figure()
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

for theta in manip_map.keys():
    M = manip_map[theta]
    F = feasi_map[theta]
    T = np.empty(len(M))
    T.fill(theta)
    scatter_3d(ax1, theta, M[0], M[1], M[2], dot_size=1.5)
    scatter_3d(ax2, theta, F[0], F[1], F[2], dot_size=1.5)

xyminmax = [-20, 20, -20, 20]
ax1.axis(xyminmax)
ax2.axis(xyminmax)
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …